# PETS CLASSIFICATION

## Libraries and extensions

Lo primero que debemos hacer antes de comenzar con la manipulacion de nuestros datos es importar todas las librerias que vayamos a utlizar.

In [1]:
from os.path import isfile, join, exists
from skimage import data, exposure
from os import listdir
import os
import torch

## Transformación y preparación del conjunto de datos

El primer problema que encontramos es que las imágenes no tienen las mismas dimensiones, por lo que debemos escalar todas las imágenes al mismo tamaño para entrenar nuestro modelo. El mejor tamaño es el mínimo de todas las imágenes, con el fin de evitar píxeles inútiles de información, ya que si aumentamos el tamaño de las imágenes pequeñas estamos haciendo que aparezcan píxeles que no existían y alteraríamos los resultados.

In [5]:
import os
from PIL import Image

def find_smallest_image_in_directory(directory):
    image_files = [file for file in os.listdir(directory) if file.lower().endswith('.jpg')]

    if not image_files:
        print(f"No images found in the directory {directory}.")
        return None, None

    min_size = float('inf')
    min_image = None

    for filename in image_files:
        image = Image.open(os.path.join(directory, filename))
        width, height = image.size
        current_size = width * height

        if current_size < min_size:
            min_size = current_size
            min_image = filename

    return min_image, min_size

base_directory = "images"

for subdirectory in os.listdir(base_directory):
    subdirectory_path = os.path.join(base_directory, subdirectory)
    
    if os.path.isdir(subdirectory_path):
        min_image, min_size = find_smallest_image_in_directory(subdirectory_path)

        if min_image is not None:
            print(f"Directorio {subdirectory}")
            print(f"Tamaño: {Image.open(os.path.join(subdirectory_path, min_image)).size}")
        else:
            print(f"No hay nada")


Directorio gatos
Tamaño: (103, 103)
Directorio images
Tamaño: (103, 103)
Directorio perros
Tamaño: (103, 103)


Como podemos comprobar, la imagen con una dimensión menor mide (103, 103). Para mantener esto de la forma más simple posible vamos a escalar todas las imágenes a un cuadrado de dimensiones (103,103). Hacemos el proceso de resize utilizando una función auxiliar para todas las imagenes del dataset.

In [4]:
import cv2

def resize_images():
    # Directorio base
    base_directory = "images"
    
    # Iterar a través de los subdirectorios
    for subdirectory in os.listdir(base_directory):
        subdirectory_path = os.path.join(base_directory, subdirectory)

        if os.path.isdir(subdirectory_path):
                
            image_files = [file for file in os.listdir(subdirectory_path) if file.lower().endswith(('.jpg', '.jpeg'))]

            if not image_files:
                print(f"No images found in the directory {subdirectory_path}.")
                continue  # Skip to the next subdirectory

            for filename in image_files:
                image_path = os.path.join(subdirectory_path, filename)
                image = cv2.imread(image_path)

                if image is not None and not image.size == 0:
                    resized_image = cv2.resize(image, (103, 103))
                    cv2.imwrite(image_path, resized_image)
                else:
                    print(f"Error loading or resizing image: {image_path}")

resize_images()

Error loading or resizing image: images\gatos\Abyssinian_34.jpg


KeyboardInterrupt: 